# How to build a simple Agent LLM App with LangGraph
* Very basic tool-using Agent LLM App with memory.

Language models can't do anything on their own; they can only create text based on what you ask them. However, LangChain allows people to build agents—think of these as smart systems or helpers— that use language models to think and decide what to do next.

Here’s how it works:
1. **Use the Language Model as a Brain**: The agent uses the language model to figure out which actions it should take, based on the information it has or what you ask it to do.
2. **Taking Action**: After deciding, the agent then goes ahead and does those actions.
3. **Learning and Adjusting**: Once the actions are done, the results can be given back to the language model. This helps the model check if everything is complete or if it needs to do something else.

So, essentially, LangChain helps turn a language model from just a tool for writing and answering into a system that can act and react, almost like a very simple robot brain.

Here we will build an agent that can interact with a search engine. You will be able to ask this agent questions, watch it calling the search tool, and have conversations with it.

## Concepts included
* [LangGraph](https://python.langchain.com/v0.2/docs/concepts/#langgraph).

LangGraph is a library created by the LangChain team for building stateful, multi-actor applications with LLMs, used to create agent and multi-agent workflows.

## Connect with the .env file located in the same directory of this notebook

In [17]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

## Connect with an LLM and start a conversation with it

* For this project, we will use OpenAI's gpt-3.5-turbo

In [18]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

## Agents
* Agents use LLMs as reasoning engines to determine which actions to take.

## Tool-using Agent
* For this basic agent we will use just one tool. In next advanced projects, you will learn how to use agents with several tools.
* **Our tool of choice will be Tavily** - a search engine. 
#### Tavily API Key
* You will need to add your Tavily API key in the .env file. Obtain a key by signing up on their [website](https://tavily.com/).

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [19]:
#pip install langgraph

#### Define tools

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [20]:
#pip install langchain-community

In [21]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=2)
search.invoke("Who are the top stars of the 2024 Eurocup?")

[{'title': 'EURO 2024 Best XI of tournament - NBC Sports',
  'url': 'https://www.nbcsports.com/soccer/news/euro-2024-best-xi-of-tournament',
  'content': '[ EURO 2024 FINAL: · Ivan Schranz, Slovakia — · Toni Kroos, Germany — · Christian Eriksen, Denmark — · Ruben Dias, Portugal — · Ferdi Kadioglu,',
  'score': 0.73299384},
 {'title': 'Euro 2024 – 5 Standout Performers - Attacking Football',
  'url': 'https://www.attackingfootball.com/euro-2024-5-standout-performers/',
  'content': 'EURO 2024: 5 STANDOUT PERFORMERS · Christoph Baumgartner (Austria) · Riccardo Calafiori (Italy) · Stanislav Lobotka (Slovakia) · Fabián Ruiz (Spain) · Zan Karnicnik',
  'score': 0.6877354}]

In [22]:
tools = [search]

* **In order to enable this model to do tool calling we use .bind_tools** to give the language model knowledge of these tools:

In [23]:
llm_with_tools = llm.bind_tools(tools)

## Create the agent
* We will be using LangGraph to construct the agent. 
* **Note that below we are passing in the origina chat model, not the llm_with_tools we built later**. That is because create_tool_calling_executor will call .bind_tools for us under the hood.

In [24]:
# from langgraph.prebuilt import chat_agent_executor

# agent_executor = chat_agent_executor.create_tool_calling_executor(llm, tools)

* In the latest version of langgraph, we have a most modern way to do this:

In [25]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

## Run the agent
* Let's first try it with .invoke():

In [26]:
from langchain_core.messages import HumanMessage

response = agent_executor.invoke({"messages": [HumanMessage(content="Where is the soccer Eurocup 2024 played?")]})

response["messages"]

[HumanMessage(content='Where is the soccer Eurocup 2024 played?', additional_kwargs={}, response_metadata={}, id='93b70289-a30a-42de-88a6-ebc9b882f662'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_XPg28ZC3shpjSB7mwsI6qdZ9', 'function': {'arguments': '{"query":"Soccer Eurocup 2024 location"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 92, 'total_tokens': 117, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BpFXNEmEKFeM9yz7gPu1HIvacVONf', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--5e7d7f25-eeff-4664-9cfb-e29919133acd-0', tool_calls=[{'name': 'tavily_search_results_json', 'a

* Now let's try it with .stream():

In [27]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="When and where will it be the 2024 Eurocup final match?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_jZissqrn9zDubwboeq2iw7Qr', 'function': {'arguments': '{"query":"2024 Eurocup final match date and location"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 96, 'total_tokens': 122, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BpFXS2mmdeKm7yYgYyCp735aiLSRF', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--a37afb8f-3daa-4ef9-a27b-95e39acae612-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 Eurocup final match date and location'}, 'id': 'call_jZissqrn9zDubwboeq2iw7Qr', 'type': 'tool_c

## Adding memory
* Adding memory in LangGraph is very similar to what we did with LangChain.

In [28]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

* Let's create our new agent with memory and set one thread_id so the agent can create a memory for each session as we did with our previous conversational RAG app:

In [29]:
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "001"}}

* Let's now try this new agent with .stream():

In [30]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who won the 2024 soccer Eurocup?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1EXbt7J3iklDk3wYq7iIzV98', 'function': {'arguments': '{"query":"2024 soccer Eurocup winner"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 91, 'total_tokens': 114, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BpFXWIFO6cRN9142a4wVKIlkvKQhS', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--093fae5f-82fc-4c60-b560-bdca7ea2012a-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 soccer Eurocup winner'}, 'id': 'call_1EXbt7J3iklDk3wYq7iIzV98', 'type': 'tool_call'}], usage_metadata={'input_t

In [31]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who were the top stars of that winner team?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="The top stars of the Spain national team that won the 2024 soccer Eurocup included players like Alvaro Morata, who played a key role in the final, helping Spain to victory. Other notable players in the team may have included rising talents and experienced players who contributed to Spain's success in the tournament.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 630, 'total_tokens': 695, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BpFXaMGNoWvXLDGsOPH3M8YS8A4Ns', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--0830a485-d8f6-4ba2-a960-6808a7a4ef6d-0', usage_metadata={'input_tokens': 630, 'output_tokens': 65, 

* Let's change the thread_id and see what happens:

In [32]:
config = {"configurable": {"thread_id": "002"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="About what soccer team we were talking?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='You were asking about the Barcelona soccer team. Would you like more information about them?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 89, 'total_tokens': 107, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BpFXcfxN55tHwkUWn4uXiJOx3RauJ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--62f9d9e1-66b6-471b-b231-46d0ca031ea4-0', usage_metadata={'input_tokens': 89, 'output_tokens': 18, 'total_tokens': 107, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


* As you can see, when we changed the threat_id we started a different conversation memory, so our app does not remember the previous interaction and instead of giving us the right answer it decided to hallucinate.